In [1]:
from ipywidgets import Label, widgets, interact, Button, Layout
import ipyleaflet as ilf
import ipywidgets as iwg
import requests
import numpy as np
from ipyevents import Event
from sidecar import Sidecar

In [2]:
def latlon_to_xy(lat, lon, factor=8):
    return int(np.floor(lon/factor)), int(np.floor(-lat/factor))
def xy_to_latlon(x,y, factor=8):
    return -y*factor, x*factor
def create_selected_polygon(xx,yy):
    c1 = (xy_to_latlon(xx, yy))
    c2 = (xy_to_latlon(xx+1, yy))
    c3 = (xy_to_latlon(xx+1, yy+1))
    c4 = (xy_to_latlon(xx, yy+1))
    temp = ilf.Polygon(
        name="selected",
        locations=[c1,c2,c3,c4],
        color="green",
        fill_opacity = 0.05,
        weight = 2,
        fill_color="green"
    )
    return temp

In [3]:
username = 'jlab'
r = requests.get("http://localhost:8889/initdb", params = {'user': username})
r.text

''

In [4]:
def cntr(change):
    old = change['old']
    new = change['new']
    yc , xc = new
    hh = float(m.layout.height.split('px')[0])
    ww = float(m.layout.width.split('px')[0])
    max_yc = -hh/(2**(m.zoom+1))
    min_xc = ww/(2**(m.zoom+1))
    max_xc = 30*8 - ww/(2**(m.zoom+1))
    min_yc = -(20*8 - hh/(2**(m.zoom+1)))    
    myc = min(yc, max_yc)
    myc = max(myc, min_yc)
    mxc = max(xc, min_xc)
    mxc = min(mxc, max_xc)
    m.center = [myc, mxc] 
    #msg.value ='{} -> {}'.format(str(old), str(new))
def handle_event(event):
    x = int(vx.value)
    y = int(vy.value)
    z = int(vz.value)
    nx = x
    ny = y
    scy = 0
    scx = 0
    if event['key'] =='w':
        ny = y-1
        scy = 2
    if event['key'] =='s':
        ny = y+1
        scy = -2
    if event['key'] =='d':
        nx = x+1
        scx = 2
    if event['key'] =='a':
        nx = x-1
        scx = -2
    if event['key'] in ('w','a','s','d'):
        for l in m.layers:
            if isinstance(l, ilf.Polygon):
                if l.name == 'selected':
                    m.remove_layer(l)
        nx = max(nx,0)
        nx = min(nx,29)
        ny = max(ny,0)
        ny = min(ny,16)
        vy.value = str(ny)
        vx.value = str(nx)
        polygon = create_selected_polygon(nx,ny)
        m.add_layer(polygon)
        label.value = 'vx={} , vy ={}, z={}'.format(x, y, z)
        r = requests.get("http://localhost:8889/info", params = {'user': username, 'x':nx, 'y':ny})
        info.value = r.text
        yc , xc = m.center
        hh = float(m.layout.height.split('px')[0])
        ww = float(m.layout.width.split('px')[0])
        max_yc = -hh/(2**(m.zoom+1))
        min_xc = ww/(2**(m.zoom+1))
        max_xc = 30*8 - ww/(2**(m.zoom+1))
        min_yc = -(20*8 - hh/(2**(m.zoom+1)))    
        myc = min(yc+scy, max_yc)
        myc = max(myc, min_yc)
        mxc = max(xc+scx, min_xc)
        mxc = min(mxc, max_xc)
        m.center = [myc, mxc] 

In [5]:
vx = widgets.Text()
vy = widgets.Text()
vz = widgets.Text()
label = Label()
info = Label()
def handle_interaction(**kwargs):
    #temp = ','.join('{0}={1!r}'.format(k,v) for k,v in kwargs.items())
    #label.value = temp
    #if kwargs.get('type') == 'keypress':
    #    label = 'key!'
    if kwargs.get('type') == 'dblclick':
        lat, lon = kwargs.get('coordinates')
        x,y = latlon_to_xy(lat,lon) 
        zoom = m.zoom
        vx.value = str(x)
        vy.value = str(y)
        vz.value = str(zoom)
        for l in m.layers:
            if isinstance(l, ilf.Polygon):
                if l.name == 'selected':
                    m.remove_layer(l)
        polygon = create_selected_polygon(x,y)
        m.add_layer(polygon)
        label.value = 'vx={} , vy ={}, z={}'.format(x, y, zoom)
        r = requests.get("http://localhost:8889/info", params = {'user': username, 'x':x, 'y':y})
        info.value = r.text

In [6]:
width = 640
height = 400
min_zoom = 3
max_zoom = 6
size_min_zoom = 2**(min_zoom+1)
center=[-height/size_min_zoom,width/size_min_zoom]
images_url = ('http://localhost:8889/?x={x}&y={y}&z={z}&inv=0')
L = ilf.TileLayer(crs=ilf.projections.Simple, url=images_url,inv=0, tile_size=256, dragging=True, no_wrap=True)
m = ilf.Map(layers=(L,), crs=ilf.projections.Simple,zoom=min_zoom, center=center, scroll_wheel_zoom=True, 
        double_click_zoom= False, bounce_at_zoom_limits=True, min_zoom=min_zoom, 
        max_zoom=max_zoom,attribution_control=False, keyboard=True )
control = ilf.FullScreenControl()
m.add_control(control)
m.layout.width = '{}px'.format(width)
m.layout.height = '{}px'.format(height)

In [7]:
display(label, info)
dd = Event(source=m, watched_events=['keydown']) #need to add map move
m.on_interaction(handle_interaction)
dd.on_dom_event(handle_event)
m.observe(cntr,names='center')
display(m)
#sc = Sidecar(title='Map')
#with sc:
#    display(m)

Label(value='')

Label(value='')

Map(center=[-25.0, 40.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [9]:
#help(sc)

In [10]:
btn = Button(description='inv', layout=Layout(width='30px'))
widget_control = ilf.WidgetControl(widget=btn, position='topleft', name='inv')
m.add_control(widget_control)
def on_button_clicked(b):
    im_url = L.url
    st = int(im_url.split('inv=')[1])
    L.url = im_url.replace('inv={}'.format(st),'inv={}'.format(st^1))
btn.on_click(on_button_clicked)

In [13]:
m.remove_control(widget_control)

In [14]:
left = L
images_url_2 = ('http://localhost:8889/?x={x}&y={y}&z={z}&inv=1')
R = ilf.TileLayer(crs="Simple", url=images_url_2,inv=1, tile_size=256, dragging=True, no_wrap=True)
right = R
control = ilf.SplitMapControl(left_layer=left, right_layer=right)
m.add_control(control)